# Similaridade Semântica entre as Resoluções
- Engine proposto para o Sistema de Recomendação
### SBERT.net - Siameses Neural Networks - Bidirectional Encoder Representations from Transformers

## Instalações

In [ ]:
#!pip install torch torchvision

In [ ]:
!pip install sentence-transformers

     |████████████████████████████████| 71kB 2.5MB/s 
     |████████████████████████████████| 1.1MB 7.6MB/s 
     |████████████████████████████████| 890kB 41.6MB/s 
     |████████████████████████████████| 1.1MB 51.7MB/s 
     |████████████████████████████████| 3.0MB 44.9MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.8-cp36-none-any.whl size=101996 sha256=cca67561b5191bc3339bb4ed248604847da3a7ba9859f2e3f25618ebb575fe9a
  Stored in directory: /root/.cache/pip/wheels/27/ec/b3/d12cc8e4daf77846db6543033d3a5642f204c0320b15945647
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=d944bf9b9228c68a2e24dba1a509885a7fa348503294a72152559050357c768c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


## Carregando amostra de Resoluções do Google Drive

In [ ]:
from google.colab import drive
arquivo = "ssr_dados_txt.csv" #@param {type:"string"}
diretorio = "/content/drive/My Drive/datasets/" #@param {type:"string"}
separador = "," #@param {type:"string"}
encode = "utf-8" #@param {type:"string"}
drive.mount('/content/drive')

import pandas as pd

arq = arquivo
dir = diretorio
sep = separador
enc = encode
resolucoes = pd.read_csv(dir+arq, sep=sep, encoding=enc)
resolucoes.shape

Mounted at /content/drive


(21, 6)

In [ ]:
resolucoes

,nro,dia,mes,ano,status,txt
0,629,16,dezembro,2013,V,"Resolução nº 629, de 16 de dezembro de 2013\nP..."
1,429,13,fevereiro,2006,R,"Resolução nº 429, de 13 de fevereiro de 2006 (..."
2,247,14,dezembro,2000,R,"Resolução nº 247, de 14 de dezembro de 2000 (R..."
3,233,25,agosto,2000,V,"Resolução nº 233, de 25 de agosto de 2000\nPub..."
4,713,11,outubro,2019,R,"Resolução nº 713, de 11 de outubro de 2019 (RE..."
5,710,28,maio,2019,V,"Resolução nº 710, de 28 de maio de 2019\nPubli..."
6,706,27,dezembro,2018,V,"Resolução nº 706, de 27 de dezembro de 2018\nP..."
7,699,28,setembro,2018,R,"Resolução nº 699, de 28 de setembro de 2018 (R..."
8,695,20,julho,2018,V,"Resolução nº 695, de 20 de julho de 2018\nPubl..."
9,598,23,outubro,2012,R,"Resolução nº 598, de 23 de outubro de 2012 (RE..."


## "Nova" Resolução
### Para fins de teste de similaridade
Foi selecionada a Resolução nro. 731 para que seja comparada com as demais 21 Resoluções

In [ ]:
nova_rel = "resolucao_731.csv" #@param {type:"string"}
new = nova_rel
dir = diretorio
sep = separador
enc = encode

resolucao731 = pd.read_csv(dir+nova_rel, sep=sep, encoding=enc)
resolucao731.shape
resolucao731

,nro,dia,mes,ano,status,txt
0,731,29,julho,2020,V,Altera a data de entrada em vigor do Regulamen...


## SBERT.net  
### Semantic Search  
https://www.sbert.net/docs/usage/semantic_search.html

In [ ]:
# carregar sentenças
sentences1 = resolucao731.txt # nova que será comparada
sentences2 = resolucoes.txt # as existentes

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('xlm-r-bert-base-nli-stsb-mean-tokens')

100%|██████████| 1.01G/1.01G [00:41<00:00, 24.7MB/s]


In [ ]:
# Corpus de Resoluções
corpus = sentences2
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# consulta sentenças da nova resolução
# extrair palavras-chave
queries = sentences1

In [ ]:
# encontrar as k setenças mais símiles no corpus
top_k = 2

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    
    # usar semantic_search para encontrar os k melhores scores
    top_results = util.semantic_search(query_embedding, 
                                       corpus_embeddings, 
                                       top_k=top_k)[0]

    print("====================== \n")
    print("Consulta: \n\n", query,)
    print("\n====================== \n")

    print("Top "+str(top_k)+" sentenças mais semelhantes no corpus:\n")
    print("======================\n")

    #for score, idx in zip(top_results[0], top_results[1]):
    #  print(corpus[idx], "(Score: %.4f)" % (score))

    for resp in (top_results):
      print(corpus[resp['corpus_id']], 
            "\n======================\n", 
            "(Score: %.4f)" % (resp['score']))
      print("======================\n")



Consulta: 

 Altera a data de entrada em vigor do Regulamento sobre Canalização e Condições de Uso de Radiofrequências para os Serviços de Radiodifusão e seus Ancilares, aprovado pela Resolução nº 721, de 11 de fevereiro de 2020.
Observação: Este texto não substitui o publicado no DOU de 30/7/2020.
O CONSELHO DIRETOR DA AGÊNCIA NACIONAL DE TELECOMUNICAÇÕES, no uso das atribuições que lhe foram conferidas pelo art. 22 da Lei nº 9.472, de 16 de julho de 1997, e pelo art. 35 do Regulamento da Agência Nacional de Telecomunicações, aprovado pelo Decreto nº 2.338, de 7 de outubro de 1997,
CONSIDERANDO a necessidade de adequar os sistemas informatizados da Agência para operacionalizar os regramentos estabelecidos no Regulamento sobre Canalização e Condições de Uso de Radiofrequências para os Serviços de Radiodifusão e seus Ancilares, aprovado pela Resolução nº 721, de 11 de fevereiro de 2020;
CONSIDERANDO os comentários recebidos decorrentes da Consulta Pública nº 59, de 9 de julho de 2020, 

In [ ]:
#from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('xlm-r-bert-base-nli-stsb-mean-tokens')

# carregar sentenças
sentences1 = resolucao731.txt # nova que será comparada
sentences2 = resolucoes.txt # as existentes


# calcular embedding (enriquecimento semântico) para as duas sentenças 
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

# calcular a similaridade por distância coseno
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

# Resultado da equiparação similaridade textual semântica
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

100%|██████████| 1.01G/1.01G [00:36<00:00, 27.4MB/s]


Altera a data de entrada em vigor do Regulamento sobre Canalização e Condições de Uso de Radiofrequências para os Serviços de Radiodifusão e seus Ancilares, aprovado pela Resolução nº 721, de 11 de fevereiro de 2020.
Observação: Este texto não substitui o publicado no DOU de 30/7/2020.
O CONSELHO DIRETOR DA AGÊNCIA NACIONAL DE TELECOMUNICAÇÕES, no uso das atribuições que lhe foram conferidas pelo art. 22 da Lei nº 9.472, de 16 de julho de 1997, e pelo art. 35 do Regulamento da Agência Nacional de Telecomunicações, aprovado pelo Decreto nº 2.338, de 7 de outubro de 1997,
CONSIDERANDO a necessidade de adequar os sistemas informatizados da Agência para operacionalizar os regramentos estabelecidos no Regulamento sobre Canalização e Condições de Uso de Radiofrequências para os Serviços de Radiodifusão e seus Ancilares, aprovado pela Resolução nº 721, de 11 de fevereiro de 2020;
CONSIDERANDO os comentários recebidos decorrentes da Consulta Pública nº 59, de 9 de julho de 2020, publicada no D

## KeyBERT
https://github.com/MaartenGr/KeyBERT

In [ ]:
!pip install keybert

  Created wheel for keybert: filename=keybert-0.1.2-cp36-none-any.whl size=8786 sha256=69d717b4619605e532a5880724e61e91c545a2bb41d90ae78d6abc20d87d7576
  Stored in directory: /root/.cache/pip/wheels/a8/a5/0c/27c2831e6f2a90be6c714b21d8d40d201134a69d806583df62
Successfully built keybert


In [ ]:
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')
stop_words = stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from keybert import KeyBERT

#model = KeyBERT('xlm-r-bert-base-nli-stsb-mean-tokens')
model = KeyBERT('xlm-r-distilroberta-base-paraphrase-v1')
keywords = model.extract_keywords(str(resolucoes.txt))
keywords

['resolução', 'dezembro', '604', '629', '2006']

In [ ]:
keywords2 = model.extract_keywords(str(resolucoes.txt), keyphrase_length=4, stop_words=stop_words,
                                   nr_candidates=20, top_n=10)
keywords2

['2013 np resolução nº',
 '11 resolução nº 604',
 'dezembro 2013 np resolução',
 '2018 npubl resolução nº',
 'dezembro 2018 np resolução',
 'resolução nº 19 dezembro',
 'resolução nº 629 16',
 'setembro 2018 resolução nº',
 'npu 16 resolução nº',
 '2018 np resolução nº']

In [ ]:
keywords3 = model.extract_keywords(str(resolucoes.txt), keyphrase_length=4, stop_words=stop_words, 
                                   use_maxsum=True, nr_candidates=20, top_n=10)
keywords3

['2019 npubli resolução nº',
 'resolução nº 604 27',
 'dezembro 2006 20 resolução',
 'npubli resolução nº 706',
 'resolução nº 429 13',
 '2003 npu 18 resolução',
 'npu 16 resolução nº',
 'resolução nº 629 16',
 'dezembro 2018 np resolução',
 'dezembro 2013 np resolução']

In [ ]:
keywords4 = model.extract_keywords(str(resolucoes.txt), keyphrase_length=4, stop_words=stop_words, 
                                   use_mmr=True, nr_candidates=20, top_n=10, diversity=0.7)
keywords4

['2013 np resolução nº',
 'nº 699 28 setembro',
 '440 12 julho 2006',
 'name txt dtype object',
 '233 25 agosto 2000',
 '20 julho 2018 npubl',
 'resolução nº 86 30',
 '27 dezembro 2010 13',
 '19 resolução nº 452',
 'dezembro 1997 rev 15']

## SBERT.net
### Semantic Textual Similarity  
https://www.sbert.net/docs/usage/semantic_textual_similarity.html

## SBERT.net  
### Paraphrase Mining  
https://www.sbert.net/docs/usage/paraphrase_mining.html

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('xlm-r-bert-base-nli-stsb-mean-tokens')

# uma lista de sentenças - possíveis dezenas de milhares de sentenças
# carregar sentenças
sentences1 = resolucao731.txt # nova que será comparada
#sentences2 = resolucoes.txt # as existentes

paraphrases1 = util.paraphrase_mining(model, sentences1)
#paraphrases2 = util.paraphrase_mining(model, sentences2)

for paraphrase1 in paraphrases1[0:10]:
    score, i, j = paraphrase1
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences1[j], score))

Delimita a consulta para as top_k

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('xlm-r-bert-base-nli-stsb-mean-tokens')

# uma lista de sentenças - possíveis dezenas de milhares de sentenças
# carregar sentenças
sentences1 = resolucao731.txt # nova que será comparada
#sentences2 = resolucoes.txt # as existentes

paraphrases = util.paraphrase_mining(model, sentences1, corpus_chunk_size=len(sentences1), top_k=1)

for paraphrase in paraphrases[0:10]:
    score, i, j = paraphrase
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences1[j], score))

## SBERT.net  
Sentence Embeddings  
https://www.sbert.net/docs/usage/computing_sentence_embeddings.html

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

#Our sentences we like to encode
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']

#Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

#Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

Sentence: This framework generates embeddings for each input sentence
Embedding: [-2.14861855e-01  3.95722806e-01  4.69087064e-01  1.36177167e-01
  6.91138115e-03  4.29976344e-01  1.03049481e+00 -8.20223093e-02
  2.20890209e-01 -8.78799796e-01 -1.11538485e-01 -1.70815572e-01
 -4.95070249e-01 -3.25745434e-01 -1.68513238e-01 -7.61577010e-01
 -3.08237493e-01  1.28200427e-01  2.28901476e-01 -8.63927841e-01
  8.01118433e-01 -5.13606608e-01 -8.88549760e-02  8.67860198e-01
  8.12778056e-01 -2.80299008e-01  1.01604223e+00  5.12090325e-01
  5.16793549e-01 -1.59422293e-01  8.63559693e-02 -7.04056501e-01
 -4.15288091e-01 -1.02641266e-02  1.39428407e-01 -7.40844477e-03
 -2.75420044e-02 -1.06805229e+00 -1.14779401e+00 -4.22716022e-01
  2.93512315e-01 -3.92946512e-01  3.17559749e-01 -1.97936110e-02
 -5.53470016e-01  4.99471044e-03 -5.46878390e-02  5.82734108e-01
 -1.38893008e+00 -4.44052637e-01 -8.77214432e-01 -1.82058170e-01
  4.69049752e-01 -6.04763389e-01 -3.85194927e-01 -1.42895263e-02
 -4.25226

## Aprendendo a utilizar SBERT com sklearn  
https://towardsdatascience.com/keyword-extraction-with-bert-724efca412ea  

- **SentenceTransformer**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

n_gram_range = (3, 3)

# extrair candidato words/phrases
count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([str(resolucoes.txt)])
candidatos = count.get_feature_names()

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('xlm-r-bert-base-nli-stsb-mean-tokens')
#model = SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1')
resolucoes_embedding = model.encode([resolucoes.txt])
candidato_embeddings = model.encode(candidatos)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

top_n = 7
distances = cosine_similarity(resolucoes_embedding, candidato_embeddings)
keywords = [candidatos[index] for index in distances.argsort()[0][-top_n:]]
keywords

['2018 resolução nº',
 'dezembro 2003 17',
 '18 resolução nº',
 '14 resolução nº',
 '17 resolução nº',
 '16 resolução nº',
 '2003 17 resolução']

In [ ]:
import numpy as np
import itertools

def max_sum_sim(resolucoes_embedding, word_embeddings, words, top_n, nr_candidates):
    # calcular a distancia e extrair as palavras-chave
    distances = cosine_similarity(resolucoes_embedding, candidato_embeddings)
    distances_candidatos = cosine_similarity(candidato_embeddings, 
                                            candidato_embeddings)

    # top_n palavras baseados nos candidatos em similaridade por coseno
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidatos[index] for index in words_idx]
    distances_candidatos = distances_candidatos[np.ix_(words_idx, words_idx)]

    # calcula a combinação de palavras que são menos semelhantes entre si
    min_sim = np.inf
    candidato = None
    for combinacao in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidatos[i][j] for i in combinacao for j in combinacao if i != j])
        if sim < min_sim:
            candidato = combinacao
            min_sim = sim

    return [words_vals[idx] for idx in candidato]

In [ ]:
max_sum_sim(resolucoes_embedding, candidato_embeddings, candidatos, 10, 20)

['16 dezembro 2013',
 '629 16 dezembro',
 'resolução nº 19',
 '15 resolução nº',
 'npubli resolução nº',
 '2018 np resolução',
 'resolução nº 629',
 '2018 npubl resolução',
 'dezembro 2003 17',
 '17 resolução nº']

In [ ]:
import numpy as np

def mmr(resolucoes_embedding, word_embeddings, words, top_n, diversity):

    # Extrair a similaridade dentro das palavras e entre as palavras do documento
    word_resolucoes_similarity = cosine_similarity(word_embeddings, resolucoes_embedding)
    word_similarity = cosine_similarity(word_embeddings)

    # Inicializar candidatos e escolher a melhor palavra-chave/frase-chave
    keywords_idx = [np.argmax(word_resolucoes_similarity)]
    candidatos_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    for _ in range(top_n - 1):
        # Extrair similaridades dentre dos candidatos
        # entre os candidatos as palavra-chave/frase-chave selecionadas
        candidato_similarities = word_resolucoes_similarity[candidatos_idx, :]
        target_similarities = np.max(word_similarity[candidatos_idx][:, keywords_idx], axis=1)

        # Calcular MMR
        mmr = (1-diversity) * candidato_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidatos_idx[np.argmax(mmr)]

        # Atualizar palavra-chave e candidatos
        keywords_idx.append(mmr_idx)
        candidatos_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [ ]:
mmr(resolucoes_embedding, candidato_embeddings, candidatos, 10, 20)

['2003 17 resolução',
 'nº 86 30',
 '440 12 julho',
 '25 agosto 2000',
 '20 dezembro 2010',
 'txt dtype object',
 '710 28 maio',
 '429 13 fevereiro',
 '2012 re 10',
 'resolução nº 351']

## UKPLab/sentence-transformers  
### Elastic Search
semantic_search_quora_elasticsearch.py  
https://github.com/UKPLab/sentence-transformers/blob/master/examples/applications/semantic_search_quora_elasticsearch.py

In [ ]:
from sentence_transformers import SentenceTransformer, util
import os
from elasticsearch import Elasticsearch, helpers
import csv
import time
import tqdm.autonotebook

es = Elasticsearch()

model = SentenceTransformer('distilbert-multilingual-nli-stsb-quora-ranking')

url = "http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv"
dataset_path = "quora_duplicate_questions.tsv"
max_corpus_size = 100000

# Download do dataset se necessário
if not os.path.exists(dataset_path):
    print("Download dataset")
    util.http_get(url, dataset_path)

# informa as sentenças únicas do arquivo
all_questions = {}
with open(dataset_path, encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
    for row in reader:
        all_questions[row['qid1']] = row['question1']
        if len(all_questions) >= max_corpus_size:
            break

        all_questions[row['qid2']] = row['question2']
        if len(all_questions) >= max_corpus_size:
            break

qids = list(all_questions.keys())
questions = [all_questions[qid] for qid in qids]

# indexação se o índice não existir
if not es.indices.exists(index="quora"):
    try:
        es_index = {
            "mappings": {
              "properties": {
                "question": {
                  "type": "text"
                },
                "question_vector": {
                  "type": "dense_vector",
                  "dims": 768
                }
              }
            }
        }

        es.indices.create(index='quora', body=es_index, ignore=[400])
        chunk_size = 500
        print("Index data - Ctrl+C pára!")
        with tqdm.tqdm(total=len(qids)) as pbar:
            for start_idx in range(0, len(qids), chunk_size):
                end_idx = start_idx+chunk_size

                embeddings = model.encode(questions[start_idx:end_idx], show_progress_bar=False)
                bulk_data = []
                for qid, question, embedding in zip(qids[start_idx:end_idx], questions[start_idx:end_idx], embeddings):
                    bulk_data.append({
                            "_index": 'quora',
                            "_id": qid,
                            "_source": {
                                "question": question,
                                "question_vector": embedding
                            }
                        })

                helpers.bulk(es, bulk_data)
                pbar.update(chunk_size)

    except:
        print("Exception durante a indexação. Continuar\n\n")

# search queries iterativas
while True:
    inp_question = input("Informar questão: ")

    encode_start_time = time.time()
    question_embedding = model.encode(inp_question)
    encode_end_time = time.time()

    #Lexical search
    bm25 = es.search(index="quora", body={"query": {"match": {"question": inp_question }}})

    #Sematic search
    sem_search = es.search(index="quora", body={
          "query": {
            "script_score": {
              "query": {
                "match_all": {}
              },
              "script": {
                "source": "cosineSimilarity(params.queryVector, doc['question_vector']) + 1.0",
                "params": {
                  "queryVector": question_embedding
                }
              }
            }
          }
        })

    print("QInformar questão: ", inp_question)
    print("Computando: embedding levou {:.3f} secs, BM25 search levou {:.3f} secs, semantic search com ES levou {:.3f} seconds".format(encode_end_time-encode_start_time, bm25['levou']/1000, sem_search['levou']/1000))

    print("BM25 resultados:")
    for hit in bm25['hits']['hits'][0:5]:
        print("\t{}".format(hit['_source']['question']))

    print("\nSemantic Search resultados:")
    for hit in sem_search['hits']['hits'][0:5]:
        print("\t{}".format(hit['_source']['question']))

    print("\n\n========\n")